In [280]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook
import sympy as sp
from IPython.display import display
from sympy import Eq, Symbol

In [281]:


# Set the reference geometry to be generated from the closed form kinematics
t_num = 15
a_num= 1000/4
b_num= 1615/4
c_num= 645/4
e_num= 286/4
gamma_num= 0.683


wb_shell = WBTessellation4PEx(
                         a = a_num,
                         b = b_num, 
                         c = c_num, 
                         e_x = e_num,
                         gamma=gamma_num, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                         n_phi_plus=2, # planned 5 
                         n_x_plus=2,  # planned 3
                         wireframe_width=5,
                        ##---- Trimming function works only in WBTessellation4P ----##
                         triW_half_cells_along_y=True,
                         triW_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
# wb_shell.interact()
orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)

Derivation of the crease nodes of the lower facet (-)

![Crease convention and nomenclature](wb_nomenclature_analytical.png)

In [282]:
# Define symbols
a, b, c, e, gamma, t = sp.symbols(r'a b c e \gamma t', real=True)

alpha = sp.pi/2 - gamma


In [283]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
}


In [284]:
# Known angles (alpha_x and alpha_z)
theta_x = sp.Symbol('theta_x', real=True)
theta_y = sp.Symbol('theta_y', real=True)  # Unknown
theta_z = sp.Symbol('theta_z', real=True)

# Trig functions
cx = sp.cos(theta_x)
sx = sp.sin(theta_x)
cy = sp.cos(theta_y)
sy = sp.sin(theta_y)
cz = sp.cos(theta_z)
sz = sp.sin(theta_z)

# Rotation matrices
R_x = sp.Matrix([
    [1, 0, 0],
    [0, cx, -sx],
    [0, sx, cx]
])
R_y = sp.Matrix([
    [cy, 0, sy],
    [0, 1, 0],
    [-sy, 0, cy]
])
R_z = sp.Matrix([
    [cz, -sz, 0],
    [sz, cz, 0],
    [0, 0, 1]
])

# Full rotation matrix (Z * Y * X)
#R = R_z @ R_y @ R_x
R_x


Matrix([
[1,            0,             0],
[0, cos(theta_x), -sin(theta_x)],
[0, sin(theta_x),  cos(theta_x)]])

In [285]:
x_folded = a+e
display(Eq(sp.Symbol(r'x^{\urcorner}'), x_folded))

Eq(x^{\urcorner}, a + e)

In [286]:
y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
display(Eq(sp.Symbol(r'y^{\urcorner}'), y_folded))

Eq(y^{\urcorner}, sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))

In [287]:
z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)
display(Eq(sp.Symbol(r'z^{\urcorner}'), z_folded))

Eq(z^{\urcorner}, a*(1 - sin(\gamma))/cos(\gamma))

In [288]:
# Calculating theta to follow the curvature of the tesselation
theta = ((z_folded - c*sp.cos(gamma))**2- y_folded**2) / ((z_folded - c*sp.cos(gamma))**2+ y_folded**2)
theta = -sp.pi +sp.acos(theta.simplify())
display(Eq(Symbol(r'\theta_x'), theta, evaluate=False)) 

Eq(\theta_x, acos((-a**2*(sin(\gamma) - 1)**2 + b**2*cos(\gamma)**2 - (a*(sin(\gamma) - 1) + c*cos(\gamma)**2)**2)/((-2*a*c*sin(\gamma) + 2*a*c - b**2 - c**2*cos(\gamma)**2)*cos(\gamma)**2)) - pi)

In [289]:
theta_x_num = np.array(theta.subs(subs_dict)).astype(np.float64).flatten().tolist()[0]
theta_x_num

-0.03208748520889472

In [290]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
    theta_x: theta_x_num,
}

subs_dict

{a: 250.0,
 b: 403.75,
 c: 161.25,
 e: 71.5,
 \gamma: 0.683,
 t: 15,
 theta_x: -0.03208748520889472}

In [291]:
O_center_lower = sp.Matrix([0, 0, 0])
display(Eq(Symbol(r'O_{\gamma-}^{+}'), O_center_lower, evaluate=False))

Eq(O_{\gamma-}^{+}, Matrix([
[0],
[0],
[0]]))

In [292]:
C_mr_lower = sp.Matrix([e, 0, 0])
C_mr_lower
display(Eq(Symbol(r'C_{\gamma-}^{\dashv}'), C_mr_lower, evaluate=False))

Eq(C_{\gamma-}^{\dashv}, Matrix([
[e],
[0],
[0]]))

In [293]:
C_mr_lower_num = np.array(C_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_mr_lower_num, orig_X_Ia[0]

([71.5, 0.0, 0.0], array([ 7.15000000e+01,  0.00000000e+00, -3.96046176e-04]))

In [294]:
C_ml_lower = sp.Matrix([-e, 0, 0])
C_ml_lower
display(Eq(Symbol(r'C_{\gamma-}^{\vdash}'), C_ml_lower, evaluate=False))

Eq(C_{\gamma-}^{\vdash}, Matrix([
[-e],
[ 0],
[ 0]]))

In [295]:
C_ml_lower_num = np.array(C_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_ml_lower_num, orig_X_Ia[1]

([-71.5, 0.0, 0.0], array([-7.15000000e+01,  0.00000000e+00, -3.96046176e-04]))

In [296]:
V_mr_lower = sp.Matrix([e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'V_{\gamma-}^{\dashv}'), V_mr_lower, evaluate=False))

Eq(V_{\gamma-}^{\dashv}, Matrix([
[c*sin(\gamma) + e],
[                0],
[    c*cos(\gamma)]]))

In [297]:
V_mr_lower_num = np.array(V_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_mr_lower_num, orig_X_Ia[6]

([173.26856909385202, 0.0, 125.07885850370505],
 array([173.26856995,   0.        , 125.07870483]))

In [298]:
V_ml_lower = sp.Matrix([-e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'V_{\gamma-}^{\dashv}'), V_ml_lower, evaluate=False))

Eq(V_{\gamma-}^{\dashv}, Matrix([
[-c*sin(\gamma) - e],
[                 0],
[     c*cos(\gamma)]]))

In [299]:
V_ml_lower_num = np.array(V_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_ml_lower_num, orig_X_Ia[7]

([-173.26856909385202, 0.0, 125.07885850370505],
 array([-173.26856995,    0.        ,  125.07870483]))

In [300]:
U_ur_lower = sp.Matrix([x_folded, y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\urcorner}'), U_ur_lower, evaluate=False))

Eq(U_{\gamma-}^{\urcorner}, Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]]))

In [301]:
U_ur_lower_num = np.array(U_ur_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_ur_lower_num, orig_X_Ia[2]

([321.5, 385.8493740048242, 118.88785926277468],
 array([321.5       , 385.84936523, 118.88827515]))

In [302]:
U_ul_lower = sp.Matrix([-x_folded, y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\ulcorner}'), U_ur_lower, evaluate=False))

Eq(U_{\gamma-}^{\ulcorner}, Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]]))

In [303]:
U_ul_lower_num = np.array(U_ul_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_ul_lower_num, orig_X_Ia[3]

([-321.5, 385.8493740048242, 118.88785926277468],
 array([-321.5       ,  385.84936523,  118.88827515]))

In [304]:
U_ll_lower = sp.Matrix([-x_folded, -y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\llcorner}'), U_ll_lower, evaluate=False))

Eq(U_{\gamma-}^{\llcorner}, Matrix([
[                                                             -a - e],
[-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [305]:
U_ll_lower_num = np.array(U_ll_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_ll_lower_num, orig_X_Ia[5]

([-321.5, -385.8493740048242, 118.88785926277468],
 array([-321.5       , -385.84936523,  118.88827515]))

In [306]:
U_lr_lower = sp.Matrix([x_folded, -y_folded, z_folded])
display(Eq(Symbol(r'U_{\gamma-}^{\llcorner}'), U_ll_lower, evaluate=False))

Eq(U_{\gamma-}^{\llcorner}, Matrix([
[                                                             -a - e],
[-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [307]:
U_lr_lower_num = np.array(U_lr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
U_lr_lower_num, orig_X_Ia[4]

([321.5, -385.8493740048242, 118.88785926277468],
 array([ 321.5       , -385.84936523,  118.88827515]))

In [308]:
delta_theta_ur = V_mr_lower - R_x * U_ll_lower
display(Eq(Symbol(r'\delta_{\theta}^{\urcorner}'), delta_theta_ur, evaluate=False))

Eq(\delta_{\theta}^{\urcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [309]:
W_ur_lower = R_x @ -C_mr_lower + delta_theta_ur
display(Eq(Symbol(r'W_{\gamma-}^{\urcorner}'), W_ur_lower, evaluate=False))

Eq(W_{\gamma-}^{\urcorner}, Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [310]:
W_ur_lower_num = np.array(W_ur_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
W_ur_lower_num, orig_X_Ia[14]

([423.268569093852, 381.8365966624425, -6.126613749715986],
 array([-423.26856909, -381.83660889,   -6.12637281]))

In [311]:
W_ul_lower = W_ur_lower.multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'W_{\gamma-}^{\ulcorner}'), W_ul_lower, evaluate=False))

Eq(W_{\gamma-}^{\ulcorner}, Matrix([
[                                                                                                                         -a - c*sin(\gamma) - e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [312]:
W_ul_lower_num = np.array(W_ul_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
W_ul_lower_num, orig_X_Ia[8]

([-423.268569093852, 381.8365966624425, -6.126613749715986],
 array([-423.26856909,  381.83660889,   -6.12637281]))

In [313]:
W_ll_lower = W_ur_lower.multiply_elementwise(sp.Matrix([-1, -1, 1]))
display(Eq(Symbol(r'W_{\gamma-}^{\llcorner}'), W_ll_lower, evaluate=False))

Eq(W_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                         -a - c*sin(\gamma) - e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [314]:
W_ll_lower_num = np.array(W_ll_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
W_ll_lower_num, orig_X_Ia[11]

([-423.268569093852, -381.8365966624425, -6.126613749715986],
 array([-816.26853858,  771.30151367,  100.3218689 ]))

In [315]:
W_lr_lower = W_ur_lower.multiply_elementwise(sp.Matrix([1, -1, 1]))
display(Eq(Symbol(r'W_{\gamma-}^{\lrcorner}'), W_lr_lower, evaluate=False))

Eq(W_{\gamma-}^{\lrcorner}, Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [316]:
W_lr_lower_num = np.array(W_lr_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
W_lr_lower_num, orig_X_Ia[17]

([423.268569093852, -381.8365966624425, -6.126613749715986],
 array([-816.26853858, -771.30151367,  100.3218689 ]))

In [317]:
H_ur_lower = delta_theta_ur
display(Eq(Symbol(r'H_{\gamma-}^{\urcorner}'), H_ur_lower, evaluate=False))

Eq(H_{\gamma-}^{\urcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [318]:
H_ur_lower_num = np.array(H_ur_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_ur_lower_num, orig_X_Ia[13]

([494.768569093852, 381.8365966624425, -6.126613749715986],
 array([-668.0371543 ,  385.84936523,  118.88729858]))

In [319]:
H_ul_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\ulcorner}'), H_ul_lower, evaluate=False))

Eq(H_{\gamma-}^{\ulcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [320]:
H_ul_lower_num = np.array(H_ul_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_ul_lower_num, orig_X_Ia[9]

([-494.768569093852, 381.8365966624425, -6.126613749715986],
 array([-566.26856909,  381.83660889,   -6.12637281]))

In [321]:
H_ll_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, -1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\llcorner}'), H_ll_lower, evaluate=False))

Eq(H_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                       -a - c*sin(\gamma) - 2*e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [322]:
H_ll_lower_num = np.array(H_ll_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_ll_lower_num, orig_X_Ia[12]

([-494.768569093852, -381.8365966624425, -6.126613749715986],
 array([-8.16268539e+02, -1.52587891e-05,  1.25078705e+02]))

In [323]:
H_lr_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([1, -1, 1]))
display(Eq(Symbol(r'H_{\gamma-}^{\llcorner}'), H_lr_lower, evaluate=False))

Eq(H_{\gamma-}^{\llcorner}, Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                -a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]]))

In [324]:
H_lr_lower_num = np.array(H_lr_lower.subs(subs_dict).evalf(), dtype=float).flatten().tolist()
H_lr_lower_num, orig_X_Ia[16]

([494.768569093852, -381.8365966624425, -6.126613749715986],
 array([-173.26859961, -771.30151367,  100.3218689 ]))

In [325]:
H_mr_lower = sp.Matrix([a + 2*e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'H_{\gamma-}^{\dashv}'), H_mr_lower, evaluate=False))

Eq(H_{\gamma-}^{\dashv}, Matrix([
[a + c*sin(\gamma) + 2*e],
[                      0],
[          c*cos(\gamma)]]))

In [326]:
H_mr_lower_num = np.array(H_mr_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
H_mr_lower_num, orig_X_Ia[15]

([494.768569093852, 0.0, 125.07885850370505],
 array([-566.26856909, -381.83660889,   -6.12637281]))

In [327]:
H_ml_lower = sp.Matrix([-a-2*e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
display(Eq(Symbol(r'H_{\gamma-}^{\vdash}'), H_ml_lower, evaluate=False))

Eq(H_{\gamma-}^{\vdash}, Matrix([
[-a - c*sin(\gamma) - 2*e],
[                       0],
[           c*cos(\gamma)]]))

In [328]:
H_ml_lower_num = np.array(H_ml_lower.subs(subs_dict)).astype(np.float64).flatten().tolist()
H_ml_lower_num, orig_X_Ia[10]

([-494.768569093852, 0.0, 125.07885850370505],
 array([-173.26859961,  771.30151367,  100.3218689 ]))

In [329]:
O_center_upper = sp.Matrix([0, 0, t])
display(Eq(Symbol(r'O_{\gamma+}^{+}'), O_center_upper, evaluate=False))

Eq(O_{\gamma+}^{+}, Matrix([
[0],
[0],
[t]]))

In [330]:
O_ml_lower_num = np.array(O_center_upper.subs(subs_dict)).astype(np.float64).flatten()
O_ml_lower_num

array([ 0.,  0., 15.])

# Compatible extrusion 

The crease nodes $U,V, W, H$ need to be extruded in z-direction, so that the compatibility is not affected.
Crease nodes that belongs to a valley are allowed to be extruded without affecting the compatibility. On the otherhand, nodes belonging to a mountain cannot be extruded without affecting the compatibility conditions


In [331]:
V_mr_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), 0, c*sp.cos(gamma)+t])
display(Eq(Symbol(r'V_{\gamma+}^{\dashv}'), V_mr_upper, evaluate=False))


Eq(V_{\gamma+}^{\dashv}, Matrix([
[c*sin(\gamma) + e - t*cot(\gamma/2 + pi/4)],
[                                         0],
[                         c*cos(\gamma) + t]]))

In [332]:
V_mr_upper_num = np.array(V_mr_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
V_mr_upper_num

[166.13529753808555, 0.0, 140.07885850370505]

In [333]:
H_mr_upper = sp.Matrix([a+2*e+c*sp.sin(gamma),0,  c*sp.cos(gamma)+t])
display(Eq(Symbol(r'H_{\gamma+}^{\dashv}'), H_mr_upper, evaluate=False))


Eq(H_{\gamma+}^{\dashv}, Matrix([
[a + c*sin(\gamma) + 2*e],
[                      0],
[      c*cos(\gamma) + t]]))

In [334]:
V_ml_upper = sp.Matrix([e+c*sp.sin(gamma)-t*sp.tan(alpha/2), 0, c*sp.cos(gamma)+t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'V_{\gamma+}^{\vdash}'), V_ml_upper, evaluate=False))

Eq(V_{\gamma+}^{\vdash}, Matrix([
[-c*sin(\gamma) - e + t*cot(\gamma/2 + pi/4)],
[                                          0],
[                          c*cos(\gamma) + t]]))

In [335]:
H_ml_upper = sp.Matrix([a+2*e+c*sp.sin(gamma),0,  c*sp.cos(gamma)+t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'H_{\gamma+}^{\vdash}'), H_ml_upper, evaluate=False))


Eq(H_{\gamma+}^{\vdash}, Matrix([
[-a - c*sin(\gamma) - 2*e],
[                       0],
[       c*cos(\gamma) + t]]))

# Extrusion of the the valley

In [336]:
C_mr_upper = sp.Matrix([e-t*sp.tan(alpha/2), 0, t])
display(Eq(Symbol(r'C_{\gamma+}^{\dashv}'), C_mr_upper, evaluate=False))


Eq(C_{\gamma+}^{\dashv}, Matrix([
[e - t*cot(\gamma/2 + pi/4)],
[                         0],
[                         t]]))

In [337]:
C_mr_upper_num = np.array(C_mr_upper.subs(subs_dict)).astype(np.float64).flatten().tolist()
C_mr_upper_num

[64.36672844423352, 0.0, 15.0]

In [338]:
C_ml_upper = sp.Matrix([e-t*sp.tan(alpha/2), 0, t]).multiply_elementwise(sp.Matrix([-1, 1, 1]))
display(Eq(Symbol(r'C_{\a}^{\vdash}'), C_ml_upper, evaluate=False))

Eq(C_{\a}^{\vdash}, Matrix([
[-e + t*cot(\gamma/2 + pi/4)],
[                          0],
[                          t]]))

In [339]:
import k3d
import numpy as np

def plot_lines_with_k3d(points, connectivity):
    """
    Plot lines with k3d given points and connectivity.

    Parameters:
    - points: (N, 3) array-like of 3D coordinates
    - connectivity: (M, 2) array-like of index pairs defining lines
    """
    points = np.array(points, dtype=np.float32)
    connectivity = np.array(connectivity, dtype=np.uint32)

    # Flatten the points into a line vertex sequence
    lines = k3d.lines(
        vertices=points,
        indices=connectivity,
        shader='simple',
        width=0.01
    )

    plot = k3d.plot()
    plot += lines
    plot.display()

# Example usage:
points = [
    C_mr_lower_num, # 0
    C_ml_lower_num, # 1
    U_ur_lower_num, # 2
    U_ul_lower_num, # 3
    U_lr_lower_num, # 4
    U_ll_lower_num, # 5
    V_mr_lower_num, # 6
    V_ml_lower_num, # 7
    W_ul_lower_num, # 8
    H_ul_lower_num, # 9
    H_ml_lower_num, # 10
    W_ll_lower_num, # 11
    H_ll_lower_num, # 12
    H_ur_lower_num, # 13
    W_ur_lower_num, # 14
    H_mr_lower_num, # 15
    H_lr_lower_num, # 16
    W_lr_lower_num, # 17
    ]

# upper facets


connectivity = [
# lower facets
[ 0,  2,  1],
[ 1,  2,  3],
[ 0,  1,  4],
[ 1,  5,  4],
[ 0,  4,  6],
[ 0,  6,  2],
[ 1,  7,  5],
[ 1,  3,  7],
[ 8,  9,  7],
[ 9, 10,  7],
[ 8,  7,  3],
[11,  7, 12],
[12,  7, 10],
[11,  5,  7],
[13, 14, 15],
[14,  6, 15],
[14,  2,  6],
[16, 15, 17],
[17, 15,  6],
[17,  6,  4],

#upper facets

]



def plot_mesh_with_k3d(points, triangles):
    points = np.array(points, dtype=np.float32)
    triangles = np.array(triangles, dtype=np.uint32)

    # Duplicate the triangles with reversed vertex order
    triangles_reversed = triangles[:, [0, 2, 1]]
    all_triangles = np.vstack([triangles, triangles_reversed])

    mesh = k3d.mesh(
        vertices=points,
        indices=all_triangles,
        wireframe=False,
        color=0x1E90FF,
        opacity=1.0
    )

    plot = k3d.plot()
    plot += mesh
    plot.display()
# Call the plotting function
plot_mesh_with_k3d(points, connectivity)

Output()

# Extrusion given thickness t for the mountains

In [ ]:
import k3d
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Parameters
grid_size = 50
initial_timesteps = 20

# Grid
X, Y = np.mgrid[-1:1:complex(grid_size), -1:1:complex(grid_size)]

# Precomputed surfaces
Z_surfaces = [np.sin(np.pi * X) * np.cos(np.pi * Y + t * 0.3) for t in range(initial_timesteps)]

# Create vertices
def make_vertices(Z):
    return np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=1).astype(np.float32)

# Create face connectivity
faces = []
for i in range(grid_size - 1):
    for j in range(grid_size - 1):
        idx = i * grid_size + j
        faces.append([idx, idx + 1, idx + grid_size])
        faces.append([idx + 1, idx + grid_size + 1, idx + grid_size])
faces = np.array(faces)

# Create plot
plot = k3d.plot()
mesh = k3d.mesh(make_vertices(Z_surfaces[0]), faces, color=0x00ff00)
plot += mesh
display(plot)

# Widgets
range_min = widgets.IntText(value=0, description='Min:')
range_max = widgets.IntText(value=initial_timesteps - 1, description='Max:')
slider = widgets.IntSlider(value=0, min=0, max=initial_timesteps - 1, step=1, description='Timestep:')
input_field = widgets.IntText(value=0, description='Input:')

# Mesh update function
def update_mesh(timestep):
    if timestep >= len(Z_surfaces):
        # Dynamically generate more surfaces as needed
        for t in range(len(Z_surfaces), timestep + 1):
            Z_surfaces.append(np.sin(np.pi * X) * np.cos(np.pi * Y + t * 0.3))

    if 0 <= timestep < len(Z_surfaces):
        mesh.vertices = make_vertices(Z_surfaces[timestep])

# Update slider constraints
def update_slider_constraints():
    min_val = range_min.value
    max_val = range_max.value

    if max_val <= min_val:
        max_val = min_val + 1
        range_max.value = max_val

    slider.min = min_val
    slider.max = max_val

    if slider.value < min_val:
        slider.value = min_val
    elif slider.value > max_val:
        slider.value = max_val

range_min.observe(lambda change: update_slider_constraints(), names='value')
range_max.observe(lambda change: update_slider_constraints(), names='value')

# Manual input: expand range, generate new timestep if needed, and update mesh
def on_input_change(change):
    val = change['new']

    # Expand range
    if val < range_min.value:
        range_min.value = val
    if val > range_max.value:
        range_max.value = val

    slider.value = val  # Set slider
    update_mesh(val)    # Update plot

# Sync slider -> input
def on_slider_change(change):
    val = change['new']
    input_field.value = val
    update_mesh(val)  # Update plot from slider

# Observe changes
input_field.observe(on_input_change, names='value')
slider.observe(on_slider_change, names='value')

# Layout
range_box = widgets.HBox([range_min, range_max])
control_box = widgets.HBox([slider, input_field])
out = widgets.interactive_output(update_mesh, {'timestep': slider})

# Display UI
display(range_box, control_box, out)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [ ]:
cos_beta = H_mr_lower[1] - H_ur_lower[1] / sp.sqrt((H_mr_lower[1] - H_ur_lower[1])**2 + (H_mr_lower[2] - H_ur_lower[2])**2)
cos_beta

-(a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))/sqrt((-a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2 + (a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2)

In [ ]:
delta_z = t / cos_beta
delta_z

-t*sqrt((-a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2 + (a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2)/(a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))

In [ ]:
delta_z = np.abs(np.array(delta_z.subs(subs_dict)))
delta_z

15.8608421846844

# Extruded geometry 

In [ ]:
import pandas as pd
import numpy as np
import k3d
from IPython.display import display
from sympy import Eq, Symbol


# Variables 

theta_x = sp.Symbol('theta_x', real=True)
a, b, c, e, gamma, t = sp.symbols(r'a b c e \gamma t', real=True)

alpha = sp.pi/2 - gamma

# Rotation
cx = sp.cos(theta_x)
sx = sp.sin(theta_x)
cy = sp.cos(theta_y)
sy = sp.sin(theta_y)
cz = sp.cos(theta_z)
sz = sp.sin(theta_z)

R_x = sp.Matrix([
    [1, 0, 0],
    [0, cx, -sx],
    [0, sx, cx]
])

# Kinematics 
x_folded = a+e
y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)

# Theta angle
theta = ((z_folded - c*sp.cos(gamma))**2- y_folded**2) / ((z_folded - c*sp.cos(gamma))**2+ y_folded**2)
theta = -sp.pi +sp.acos(theta.simplify())

# Calculate crease nodes from the waterbomb module at the lower surface analytically 
O_center_lower = sp.Matrix([0, 0, 0])
C_mr_lower = sp.Matrix([e, 0, 0])
C_ml_lower = sp.Matrix([-e, 0, 0])
V_mr_lower = sp.Matrix([e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
V_ml_lower = sp.Matrix([-e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])
U_ur_lower = sp.Matrix([x_folded, y_folded, z_folded])
U_ul_lower = sp.Matrix([-x_folded, y_folded, z_folded])
U_ll_lower = sp.Matrix([-x_folded, -y_folded, z_folded])
U_lr_lower = sp.Matrix([x_folded, -y_folded, z_folded])
delta_theta_ur = V_mr_lower - R_x * U_ll_lower
W_ur_lower = R_x @ -C_mr_lower + delta_theta_ur
W_ul_lower = W_ur_lower.multiply_elementwise(sp.Matrix([-1, 1, 1]))
W_ll_lower = W_ur_lower.multiply_elementwise(sp.Matrix([-1, -1, 1]))
W_lr_lower = W_ur_lower.multiply_elementwise(sp.Matrix([1, -1, 1]))
H_ur_lower = delta_theta_ur
H_ul_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, 1, 1]))
H_ll_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([-1, -1, 1]))
H_lr_lower = delta_theta_ur.multiply_elementwise(sp.Matrix([1, -1, 1]))
H_mr_lower = sp.Matrix([a + 2*e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
H_ml_lower = sp.Matrix([-a-2*e-c*sp.sin(gamma), 0, c*sp.cos(gamma)])



# Support normal to the facet

In [ ]:
# Given C_mr_lower_num, U_ur_lower_num, U_ul_lower_num, calculate the unit normal vector
def calculate_normal_vector(p1, p2, p3):
    # Convert points to numpy arrays
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)

    # Calculate two vectors in the plane
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate the cross product to get the normal vector
    normal_vector = np.cross(v1, v2)

    # Normalize the normal vector
    normal_vector /= np.linalg.norm(normal_vector)

    return normal_vector
# Example points
p1 = np.array(C_mr_lower_num)
p2 = np.array(U_ul_lower_num)
p3 = np.array(U_ur_lower_num)
# Calculate the normal vector
normal_vector = calculate_normal_vector(p1, p2, p3)
normal_vector

array([ 0.        ,  0.29445909, -0.95566408])